# Will's cone detection notebook

## Purpose

I'm not very good at java tbh, so I'll do my code things here.

In [244]:
from functools import reduce
import cv2
import numpy as np
import matplotlib.pyplot as plt
from num2words import num2words

# grab relative path for loading images
import os
notebook_path = os.path.abspath("Notebook.ipynb")

def compose(*f_arg_pairs):
    def cr(*fs):
        pair = fs[0]
        if len(fs) == 1:
            return lambda x:pair[0](x,*pair[1])
        return lambda x:pair[0](cr(*fs[1:])(x),*pair[1])
    return cr(*reversed(f_arg_pairs))

digitalsum = compose(
    (int,()),
    (str,()),
    (list,()),
    (lambda x: (int(n) for n in x),()),
    (sum, ()),
)

print(digitalsum(314159265358979323))


def add_image(*dispImage):
    fig = plt.figure(figsize=(6*len(dispImage),8))
    for index, img in enumerate(dispImage):
        fig.add_subplot(1, len(dispImage), index+1)
        # showing image
        plt.imshow(img)
        plt.axis('off')
        plt.title(num2words(index+1,True,to="ordinal").capitalize())


85


## Image processing functions

In [433]:
def flood_from_pix(floodImg: np.ndarray, pixel: tuple = ()):
    h, w = floodImg.shape[:2]
    if not pixel: pixel = (w//2, h//2)
    mask = np.zeros((h+2,w+2), np.uint8)
    flat = floodImg.astype(np.uint8)
    cv2.floodFill(flat, mask, pixel, 255)
    return cv2.threshold(flat, 254, 255, cv2.THRESH_BINARY)[1]

def resize_image(dimImage: np.ndarray, newsize: tuple = ()):
    print(dims:=dimImage.shape)
    if newsize:
        rowedge = (dims[0]-newsize[0])//2
        coledge = (dims[1]-newsize[1])//2

        return dimImage[rowedge:dims[0]-rowedge,coledge:dims[1]-coledge]

    return dimImage

def tri_stack(stackImg: np.ndarray):
    return np.dstack([stackImg,stackImg,stackImg])

def extract_channel(extrImg: np.ndarray, channel: int = 0):
    return tri_stack(extrImg[:,:,channel])

def point(pointImg: np.ndarray, xy: tuple, clr: tuple):
    return cv2.circle(pointImg, xy, radius=5, color=clr, thickness=-1)

## Shape Pipeline

This pipeline takes a cone with a shape on it, and returns a mask over the shape.

In [456]:
from pickle import dump

color_pipeline = lambda img, seed: compose(
        (extract_channel, (0,)),
        # 200 is min, 250 is max. If theres too much bg noise, turn up 200. If the shape gets cut, turn up 250.
        (cv2.Canny, (200, 250)),
        (cv2.blur, ((7, 7),)),
        (flood_from_pix, (seed,)),
        (cv2.Canny, (200, 250)),
    )(img)

image = cv2.imread("Images/Shapes V2/Normal/Triangle.png", cv2.IMREAD_UNCHANGED)
# webcam = cv2.VideoCapture(0)
# image = webcam.read()[1]
# webcam.release()

prep_pipeline = compose(
    (cv2.flip, (1,)),
    (cv2.cvtColor, (cv2.COLOR_RGB2BGR,)),
    # (resize_image, ((480,360),)),
)

image = prep_pipeline(image)

cnvimage = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
cnvimage = color_pipeline(image,(240,240))
# cnvimage = cv2.cvtColor(cnvimage, cv2.COLOR_HSV2RGB)
# add_image(image,cnvimage)
# cnvimage = cv2.cvtColor(cnvimage, cv2.COLOR_RGB2GRAY)

# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
# cv2.imwrite("Images/Shapes V2/Normal/Pentagon.png",image)
# cv2.imwrite("Images/Shapes V2/Processed/Pentagon.png",cnvimage)

dump(cnvimage, open("temp/cnvimage",'wb+'))

### Grab edges

Take the edges, find the contours, and then approximate as a polygon.

In [ ]:
from pickle import load


cnvimage = load(open("temp/cnvimage",'rb'))

